In [ ]:
! pip install groq streamlit
! pip install streamlit-ace

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.9 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from groq import Groq
from streamlit_ace import st_ace
from datetime import datetime
import time

# --- Constants ---
THEMES = ["monokai", "github", "twilight"]
LANGUAGES = ["python", "javascript", "java", "c", "cpp"]

# --- Groq Client Setup ---
client = Groq(api_key="")  # Replace with your actual key

# --- Page Config ---
st.set_page_config(
    page_title="🔍 CodeMedic AI",
    page_icon="🤖",
    layout="centered",
    initial_sidebar_state="expanded"
)

# --- Session State ---
if 'history' not in st.session_state:
    st.session_state.history = []
if 'processing' not in st.session_state:
    st.session_state.processing = False

# --- UI Components ---
def gradient_text(text):
    return f"""
    <h1 style="
        background: linear-gradient(45deg, #6EE7B7, #3B82F6);
        -webkit-background-clip: text;
        background-clip: text;
        color: transparent;
        text-align: center;
    ">{text}</h1>
    """

# --- Sidebar ---
with st.sidebar:
    st.markdown(gradient_text("CodeMedic AI"), unsafe_allow_html=True)
    st.markdown("---")

    with st.expander("⚙️ Settings"):
        selected_theme = st.selectbox("Editor Theme", THEMES, index=0)
        selected_lang = st.selectbox("Code Language", LANGUAGES, index=0)
        model_temp = st.slider("🧠 AI Creativity", 0.0, 1.0, 0.7)

    st.markdown("---")
    st.button("🧹 Clear History", on_click=lambda: st.session_state.history.clear())
    st.markdown(f"<small>Session Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</small>", unsafe_allow_html=True)

# --- Main Title ---
st.markdown(gradient_text("Code Diagnostics Suite"), unsafe_allow_html=True)
st.markdown("<div style='text-align: center; margin-bottom: 30px;'>AI-powered code analysis with deep error detection and fix generation</div>", unsafe_allow_html=True)

# --- Code Editor ---
with st.container(border=True):
    code = st_ace(
        placeholder=f"Enter {selected_lang} code...",
        language=selected_lang,
        theme=selected_theme,
        height=400,
        key=f"ace_{selected_lang}",
        wrap=True
    )

# --- Analysis Controls ---
col1, col2 = st.columns([1, 3])
with col1:
    analyze_btn = st.button("🚀 Analyze Code", use_container_width=True)
with col2:
    if st.session_state.processing:
        st.warning("AI is analyzing code...")

# --- Analysis Logic ---
if analyze_btn and not st.session_state.processing:
    if not code.strip():
        st.error("⚠️ Please enter valid code")
    else:
        st.session_state.processing = True
        try:
            with st.spinner("🔍 Deep code analysis in progress..."):
                start_time = time.time()
                response = client.chat.completions.create(
                    messages=[{
                        "role": "system",
                        "content": (
                            "You are an expert static code analyzer. Your only job is to analyze code for:\n"
                            "1. Syntax validation\n2. Logical error detection\n"
                            "3. Security vulnerabilities\n4. Optimization suggestions\n\n"
                            "IMPORTANT:\n"
                            "- Only respond with code-related analysis.\n"
                            "- Do NOT respond to general questions.\n"
                            "- If the user input is not code or code-related, reply with:\n"
                            "  '⚠️ Please input code to analyze. I only respond to code-related requests.'"
                        )
                    }, {
                        "role": "user",
                        "content": code
                    }],
                    model="llama3-70b-8192",
                    temperature=model_temp
                )
                analysis = response.choices[0].message.content
                duration = time.time() - start_time

                st.session_state.history.append({
                    "timestamp": datetime.now().isoformat(),
                    "code": code,
                    "analysis": analysis,
                    "language": selected_lang,
                    "duration": f"{duration:.2f}s"
                })

            st.success("✅ Analysis Complete")
            st.markdown("### 📝 Analysis Report")
            st.markdown(analysis)

        except Exception as e:
            st.error(f"❌ Analysis Failed: {str(e)}")

        finally:
            st.session_state.processing = False

# --- History Section ---
if st.session_state.history:
    st.markdown("## 📜 Analysis History")
    for entry in reversed(st.session_state.history):
        with st.container(border=True):
            cols = st.columns([1, 3, 1])
            cols[0].markdown(f"**Language**: {entry['language']}")
            cols[1].markdown(f"**Time**: {entry['timestamp']}")
            cols[2].markdown(f"**Duration**: {entry['duration']}")

            st.markdown("**Code:**")
            st.code(entry['code'], language=entry['language'])

            st.markdown("**Analysis:**")
            st.markdown(entry['analysis'])

# --- Footer ---
st.markdown("---")
st.markdown("""
<div style="text-align: center; color: #6B7280;">
    <div style="display: flex; justify-content: center; gap: 15px; margin-bottom: 10px;">
        <a href="https://groq.com" style="color: #3B82F6; text-decoration: none;">Powered by Groq</a>
        <span>•</span>
        <a href="#" style="color: #3B82F6; text-decoration: none;">Documentation</a>
        <span>•</span>
        <a href="#" style="color: #3B82F6; text-decoration: none;">Feedback</a>
    </div>
    <div>Made with ❤️ by AI Code Experts</div>
</div>
""", unsafe_allow_html=True)


2025-06-02 15:04:10.259 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 15:04:11.190 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 15:04:11.196 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 15:04:11.197 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-06-02 15:04:11.199 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 15:04:11.200 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

DeltaGenerator()

In [ ]:
!wget -q -O - ipv4.icanhazip.com


34.169.165.72


In [ ]:
 !streamlit run app.py & npx localtunnel --port 8501




⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.165.72:8501

⠹⠸⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://fair-regions-dress.loca.lt
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:16619 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙